In [1]:
import os
import traceback
import time
import sqlite3
from pprint import pprint
from itertools import product
from collections import defaultdict
import numpy as np
from abracadabra import settings
from abracadabra.storage import setup_db
from abracadabra.recognise import register_directory, recognise_song

In [2]:
# Parameters to search

SAMPLE_RATE = [11025, 22050]
POINT_EFFICIENCY = [0.6, 0.8]
TARGET_T = [1.8, 2, 2.2]
TARGET_F = [2000, 4000, 6000]
FFT_WINDOW_SIZE = [0.1, 0.2, 0.4]

In [3]:
experiment_db_path = "temp_experiment.db"
test_path = "test/"

def clean_for_experiment():
    try:
        os.remove(experiment_db_path)
    except OSError:
        pass

def run_experiment(parameter_set):
    # configure parameters
    settings.configure(**parameter_set)
    settings.configure(DB_PATH=experiment_db_path)
    
    # set up DB
    setup_db()
    
    # register what's in music
    register_start_time = time.time()
    register_directory("/home/cameron/Music")
    register_time = time.time() - register_start_time
    
    # run test suite
    wrong_songs = []
    correct = 0
    total = 0
    
    test_start_time = time.time()
    for root, _, files in os.walk(test_path):
        for f in files:
            if f.split('.')[-1] != 'wav':
                continue
            file_path = os.path.join(root, f)
            result = recognise_song(file_path)
            result_title = result[2]
            if result_title.replace(" ", "").lower() in f:
                correct += 1
            else:
                wrong_songs.append(file_path)
            total += 1
    test_time = time.time() - test_start_time
    
    db_size_mb = os.stat(experiment_db_path).st_size / (1024*1024)
    
    return register_time, test_time, correct/total, wrong_songs, db_size_mb

```python
sample_parameter_set = {
    'SAMPLE_RATE': 22050,
    'PEAK_BOX_SIZE': 40,
    'POINT_EFFICIENCY': 0.6,
    'TARGET_T': 1.6,
    'TARGET_F': 2000,
    'FFT_WINDOW_SIZE': 0.1
}

# the below takes 233s to register, and that's a lower bound
run_experiment(sample_parameter_set)
```

Result:

```
(233.28253030776978,
 3.6409788131713867,
 0.4666666666666667,
 ['test/aha/thebluesky3.wav',
  ...
  'test/chriscornell/wideawake3.wav'],
 60.953125)
```
 
Testing num workers:

```python
sample_parameter_set = {
    'SAMPLE_RATE': 11025,
    'PEAK_BOX_SIZE': 40,
    'POINT_EFFICIENCY': 0.6,
    'TARGET_T': 1.6,
    'TARGET_F': 2000,
    'FFT_WINDOW_SIZE': 0.1
}

clean_for_experiment()
settings.configure(NUM_WORKERS=24)
run_experiment(sample_parameter_set)

# NUM_WORKERS = 16 => 94s register
# NUM_WORKERS = 20 => 91s register
# NUM_WORKERS = 24 => 90s register
```

In [4]:
results = [({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (508.92338037490845,
   12.44532561302185,
   0.9185185185185185,
   ['test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/ab/stillremains2.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/bleeditdry1.wav'],
   646.4375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (522.9520421028137,
   6.280886888504028,
   0.9185185185185185,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/openyoureyes1.wav',
    'test/chriscornell/groundzero1.wav'],
   594.94921875)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.1},
  (687.2617120742798,
   26.502663135528564,
   0.9703703703703703,
   ['test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/ab/wonderfullife1.wav'],
   947.44921875)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.2},
  (639.6174926757812,
   19.425877809524536,
   0.9333333333333333,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/openyoureyes1.wav'],
   876.2890625)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (637.626852273941,
   14.751646518707275,
   0.9259259259259259,
   ['test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/ab/lover2.wav',
    'test/ab/stillremains3.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/cryofachilles9.wav'],
   723.3125)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (630.604278087616,
   7.088295936584473,
   0.9333333333333333,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/openyoureyes1.wav'],
   647.33984375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.1},
  (707.669088602066,
   32.65785598754883,
   0.9481481481481482,
   ['test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/ab/stillremains3.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/cryofachilles9.wav'],
   1057.1875)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.2},
  (665.3205983638763,
   7.393085479736328,
   0.9407407407407408,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/stillremains2.wav',
    'test/ab/openyoureyes1.wav'],
   954.2890625)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (700.1374673843384,
   16.93176794052124,
   0.0,
   ['test/aha/thebluesky3.wav',
    'test/aha/takeonme5.wav',
    'test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/takeonme2.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/loveisreason1.wav',
    'test/aha/manhattanskyline1.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/andyoutellme3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/takeonme4.wav',
    'test/aha/manhattanskyline3.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/thebluesky2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited1.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/lover2.wav',
    'test/ab/lover4.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/makeitright2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/theuninvited9.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/lover6.wav',
    'test/ab/theuninvited6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/theuninvited10.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/bleeditdry7.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/burnitdown1.wav',
    'test/ab/addictedtopain6.wav',
    'test/ab/theuninvited3.wav',
    'test/ab/lover5.wav',
    'test/ab/brokenwings2.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/findthereal3.wav',
    'test/ab/findthereal1.wav',
    'test/ab/bleeditdry2.wav',
    'test/ab/brokenwings1.wav',
    'test/ab/theuninvited5.wav',
    'test/ab/theuninvited4.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/onedayremains1.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry9.wav',
    'test/ab/lover1.wav',
    'test/ab/burnitdown2.wav',
    'test/ab/addictedtopain8.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/ab/wonderfullife2.wav',
    'test/ab/bleeditdry3.wav',
    'test/chriscornell/iamthehighway1.wav',
    'test/chriscornell/iamthehighway3.wav',
    'test/chriscornell/ashopeandpromisefade1.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/iamthehighway2.wav',
    'test/chriscornell/thekeeper3.wav',
    'test/chriscornell/thankyou1.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/thekeeper1.wav',
    'test/chriscornell/thekeeper2.wav',
    'test/chriscornell/groundzero2.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/thankyou2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/iamthehighway4.wav',
    'test/chriscornell/wideawake1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav',
    'test/chriscornell/wideawake3.wav'],
   1291.30859375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (679.1809005737305,
   7.169081926345825,
   0.5333333333333333,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/andyoutellme3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/thebluesky2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/theuninvited9.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/stillremains1.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/theuninvited5.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry9.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   742.2109375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 1.8,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.1},
  (829.4966192245483,
   23.28786849975586,
   0.0,
   ['test/aha/thebluesky3.wav',
    'test/aha/takeonme5.wav',
    'test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/takeonme2.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/loveisreason1.wav',
    'test/aha/manhattanskyline1.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/andyoutellme3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/takeonme4.wav',
    'test/aha/manhattanskyline3.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/thebluesky2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited1.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/lover2.wav',
    'test/ab/lover4.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/makeitright2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/theuninvited9.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/lover6.wav',
    'test/ab/theuninvited6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/theuninvited10.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/bleeditdry7.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/burnitdown1.wav',
    'test/ab/addictedtopain6.wav',
    'test/ab/theuninvited3.wav',
    'test/ab/lover5.wav',
    'test/ab/brokenwings2.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/findthereal3.wav',
    'test/ab/findthereal1.wav',
    'test/ab/bleeditdry2.wav',
    'test/ab/brokenwings1.wav',
    'test/ab/theuninvited5.wav',
    'test/ab/theuninvited4.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/onedayremains1.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry9.wav',
    'test/ab/lover1.wav',
    'test/ab/burnitdown2.wav',
    'test/ab/addictedtopain8.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/ab/wonderfullife2.wav',
    'test/ab/bleeditdry3.wav',
    'test/chriscornell/iamthehighway1.wav',
    'test/chriscornell/iamthehighway3.wav',
    'test/chriscornell/ashopeandpromisefade1.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/iamthehighway2.wav',
    'test/chriscornell/thekeeper3.wav',
    'test/chriscornell/thankyou1.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/thekeeper1.wav',
    'test/chriscornell/thekeeper2.wav',
    'test/chriscornell/groundzero2.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/thankyou2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/iamthehighway4.wav',
    'test/chriscornell/wideawake1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav',
    'test/chriscornell/wideawake3.wav'],
   1678.1484375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 1.8,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.2},
  (754.2877147197723,
   8.853631734848022,
   0.6518518518518519,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/stillremains1.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   1064.45703125)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 2,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (708.2246479988098,
   18.47409725189209,
   0.0,
   ['test/aha/thebluesky3.wav',
    'test/aha/takeonme5.wav',
    'test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/takeonme2.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/loveisreason1.wav',
    'test/aha/manhattanskyline1.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/andyoutellme3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/takeonme4.wav',
    'test/aha/manhattanskyline3.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/thebluesky2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited1.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/lover2.wav',
    'test/ab/lover4.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/makeitright2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/theuninvited9.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/lover6.wav',
    'test/ab/theuninvited6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/theuninvited10.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/bleeditdry7.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/burnitdown1.wav',
    'test/ab/addictedtopain6.wav',
    'test/ab/theuninvited3.wav',
    'test/ab/lover5.wav',
    'test/ab/brokenwings2.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/findthereal3.wav',
    'test/ab/findthereal1.wav',
    'test/ab/bleeditdry2.wav',
    'test/ab/brokenwings1.wav',
    'test/ab/theuninvited5.wav',
    'test/ab/theuninvited4.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/onedayremains1.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry9.wav',
    'test/ab/lover1.wav',
    'test/ab/burnitdown2.wav',
    'test/ab/addictedtopain8.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/ab/wonderfullife2.wav',
    'test/ab/bleeditdry3.wav',
    'test/chriscornell/iamthehighway1.wav',
    'test/chriscornell/iamthehighway3.wav',
    'test/chriscornell/ashopeandpromisefade1.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/iamthehighway2.wav',
    'test/chriscornell/thekeeper3.wav',
    'test/chriscornell/thankyou1.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/thekeeper1.wav',
    'test/chriscornell/thekeeper2.wav',
    'test/chriscornell/groundzero2.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/thankyou2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/iamthehighway4.wav',
    'test/chriscornell/wideawake1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav',
    'test/chriscornell/wideawake3.wav'],
   1382.40625)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 2,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (663.6730189323425,
   8.06558084487915,
   0.5481481481481482,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/thebluesky2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/theuninvited9.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/stillremains1.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry9.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/wonderfullife2.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   803.4140625)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 2,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.1},
  (813.2995767593384,
   27.59835457801819,
   0.007407407407407408,
   ['test/aha/thebluesky3.wav',
    'test/aha/takeonme5.wav',
    'test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/takeonme2.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/loveisreason1.wav',
    'test/aha/manhattanskyline1.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/andyoutellme3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/takeonme4.wav',
    'test/aha/manhattanskyline3.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/thebluesky2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited1.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/lover2.wav',
    'test/ab/lover4.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/makeitright2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/theuninvited9.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/lover6.wav',
    'test/ab/theuninvited6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/theuninvited10.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/bleeditdry7.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/burnitdown1.wav',
    'test/ab/addictedtopain6.wav',
    'test/ab/theuninvited3.wav',
    'test/ab/lover5.wav',
    'test/ab/brokenwings2.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/findthereal3.wav',
    'test/ab/findthereal1.wav',
    'test/ab/bleeditdry2.wav',
    'test/ab/brokenwings1.wav',
    'test/ab/theuninvited5.wav',
    'test/ab/theuninvited4.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/onedayremains1.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry9.wav',
    'test/ab/lover1.wav',
    'test/ab/burnitdown2.wav',
    'test/ab/addictedtopain8.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/wonderfullife2.wav',
    'test/ab/bleeditdry3.wav',
    'test/chriscornell/iamthehighway1.wav',
    'test/chriscornell/iamthehighway3.wav',
    'test/chriscornell/ashopeandpromisefade1.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/iamthehighway2.wav',
    'test/chriscornell/thekeeper3.wav',
    'test/chriscornell/thankyou1.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/thekeeper1.wav',
    'test/chriscornell/thekeeper2.wav',
    'test/chriscornell/groundzero2.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/thankyou2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/iamthehighway4.wav',
    'test/chriscornell/wideawake1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav',
    'test/chriscornell/wideawake3.wav'],
   1805.37109375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 1,
   'TARGET_T': 2,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.2},
  (768.8747231960297,
   9.86514163017273,
   0.6666666666666666,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/andyoutellme2.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/theswingofthings3.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/theuninvited11.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/stillremains2.wav',
    'test/ab/bleeditdry8.wav',
    'test/ab/makeitright1.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/theuninvited7.wav',
    'test/ab/stillremains1.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/theuninvited2.wav',
    'test/ab/openyoureyes1.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/chriscornell/callmeadog3.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   1154.0390625)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (1795.189745426178,
   21.900994777679443,
   0.5481481481481482,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/loveisreason1.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/lover6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/allhopeisgone1.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/lover5.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/groundzero1.wav'],
   1328.13671875)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (1775.3928315639496,
   11.261643886566162,
   0.9851851851851852,
   ['test/aha/huntinghighandlow3.wav', 'test/aha/huntinghighandlow2.wav'],
   1168.93359375)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.1},
  (1947.6467032432556,
   32.76082468032837,
   0.4222222222222222,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/theswingofthings4.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/october2.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/takeonme4.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/makeitright2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/lover6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/addictedtopain6.wav',
    'test/ab/theuninvited3.wav',
    'test/ab/lover5.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/findthereal1.wav',
    'test/ab/bleeditdry2.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/ab/wonderfullife2.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/groundzero2.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/thankyou2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/wideawake1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   2002.6171875)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.2},
  (1881.8908104896545,
   12.38567066192627,
   0.9777777777777777,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav'],
   1778.28125)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (1765.9685530662537,
   25.3468816280365,
   0.5259259259259259,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/takeonme2.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/andyoutellme3.wav',
    'test/aha/october3.wav',
    'test/aha/loveisreason2.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/october2.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/lover6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/lover5.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/cryofachilles8.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   1497.80859375)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (1777.268835067749,
   11.99790096282959,
   0.9777777777777777,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav'],
   1274.5078125)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.1},
  (1974.4253044128418,
   40.22332453727722,
   0.4222222222222222,
   ['test/aha/thebluesky3.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/trainofthought3.wav',
    'test/aha/takeonme3.wav',
    'test/aha/trainofthought2.wav',
    'test/aha/softrainsofapril3.wav',
    'test/aha/loveisreason3.wav',
    'test/aha/october3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/trainofthought1.wav',
    'test/aha/october1.wav',
    'test/aha/takeonme1.wav',
    'test/aha/scoundreldays2.wav',
    'test/aha/scoundreldays3.wav',
    'test/aha/october2.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/takeonme4.wav',
    'test/aha/trainofthought4.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/aha/softrainsofapril2.wav',
    'test/ab/stillremains3.wav',
    'test/ab/cryofachilles3.wav',
    'test/ab/makeitright2.wav',
    'test/ab/cryofachilles10.wav',
    'test/ab/addictedtopain2.wav',
    'test/ab/lover6.wav',
    'test/ab/cryofachilles6.wav',
    'test/ab/makeitright1.wav',
    'test/ab/onedayremains3(quiet).wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/findthereal2.wav',
    'test/ab/peaceisbroken3.wav',
    'test/ab/addictedtopain3.wav',
    'test/ab/cryofachilles5.wav',
    'test/ab/addictedtopain7.wav',
    'test/ab/addictedtopain5.wav',
    'test/ab/peaceisbroken2.wav',
    'test/ab/makeitright3.wav',
    'test/ab/lover7.wav',
    'test/ab/bleeditdry6.wav',
    'test/ab/cryofachilles4.wav',
    'test/ab/addictedtopain6.wav',
    'test/ab/theuninvited3.wav',
    'test/ab/lover5.wav',
    'test/ab/brokenwings2.wav',
    'test/ab/stillremains1.wav',
    'test/ab/peaceisbroken5.wav',
    'test/ab/metalingus2.wav',
    'test/ab/peaceisbroken6.wav',
    'test/ab/lover3.wav',
    'test/ab/cryofachilles2.wav',
    'test/ab/peaceisbroken4.wav',
    'test/ab/bleeditdry4.wav',
    'test/ab/findthereal1.wav',
    'test/ab/bleeditdry2.wav',
    'test/ab/cryofachilles1.wav',
    'test/ab/onedayremains2(loud).wav',
    'test/ab/addictedtopain4.wav',
    'test/ab/bleeditdry5.wav',
    'test/ab/lover8.wav',
    'test/ab/peaceisbroken1.wav',
    'test/ab/openyoureyes2.wav',
    'test/ab/bleeditdry1.wav',
    'test/ab/addictedtopain1.wav',
    'test/ab/cryofachilles9.wav',
    'test/ab/cryofachilles7.wav',
    'test/ab/metalingus1.wav',
    'test/ab/wonderfullife2.wav',
    'test/chriscornell/ashopeandpromisefade2.wav',
    'test/chriscornell/ashopeandpromisefade3.wav',
    'test/chriscornell/thankyou3.wav',
    'test/chriscornell/callmeadog2.wav',
    'test/chriscornell/groundzero2.wav',
    'test/chriscornell/wideawake2.wav',
    'test/chriscornell/callmeadog1.wav',
    'test/chriscornell/wideawake1.wav',
    'test/chriscornell/groundzero3.wav',
    'test/chriscornell/groundzero1.wav'],
   2248.9140625)),
 ({'SAMPLE_RATE': 44100,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 2,
   'TARGET_F': 6000,
   'FFT_WINDOW_SIZE': 0.2},
  (1957.8937346935272,
   15.596041440963745,
   0.9777777777777777,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav'],
   1939.36328125))]

In [5]:
def result_already_calculated(params):
    for result in results:
        if params == result[0]:
            return True
    return False

for comb in product(SAMPLE_RATE, POINT_EFFICIENCY, TARGET_T, TARGET_F, FFT_WINDOW_SIZE):
    exp_parameters = {
        'SAMPLE_RATE': comb[0],
        'POINT_EFFICIENCY': comb[1],
        'TARGET_T': comb[2],
        'TARGET_F': comb[3],
        'FFT_WINDOW_SIZE': comb[4]
    }
    if result_already_calculated(exp_parameters):
        continue
    pprint(exp_parameters)
    completed = False
    while not completed:
        try:
            clean_for_experiment()
            result = exp_parameters, run_experiment(exp_parameters)
            print(f"Acc: {result[1][2]}, Time: {result[1][0] + result[1][1]}, DB size {result[1][-1]}")
            results.append(result)
            completed = True
        except sqlite3.OperationalError as e:
            print("Failed")
            print(traceback.format_exc())

{'FFT_WINDOW_SIZE': 0.1,
 'POINT_EFFICIENCY': 0.6,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 2000,
 'TARGET_T': 1.8}
Acc: 0.5777777777777777, Time: 144.55441999435425, DB size 106.15625
{'FFT_WINDOW_SIZE': 0.2,
 'POINT_EFFICIENCY': 0.6,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 2000,
 'TARGET_T': 1.8}
Acc: 0.6592592592592592, Time: 155.0715572834015, DB size 107.93359375
{'FFT_WINDOW_SIZE': 0.4,
 'POINT_EFFICIENCY': 0.6,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 2000,
 'TARGET_T': 1.8}
Acc: 0.6444444444444445, Time: 164.47072005271912, DB size 116.0234375
{'FFT_WINDOW_SIZE': 0.1,
 'POINT_EFFICIENCY': 0.6,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 4000,
 'TARGET_T': 1.8}
Acc: 0.6518518518518519, Time: 188.36431622505188, DB size 198.13671875
{'FFT_WINDOW_SIZE': 0.2,
 'POINT_EFFICIENCY': 0.6,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 4000,
 'TARGET_T': 1.8}
Acc: 0.7481481481481481, Time: 182.1498625278473, DB size 195.8046875
{'FFT_WINDOW_SIZE': 0.4,
 'POINT_EFFICIENCY': 0.6,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 4000,

Acc: 0.6666666666666666, Time: 177.05954837799072, DB size 192.6171875
{'FFT_WINDOW_SIZE': 0.2,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 2000,
 'TARGET_T': 2.2}
Acc: 0.7555555555555555, Time: 172.62787437438965, DB size 182.7734375
{'FFT_WINDOW_SIZE': 0.4,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 2000,
 'TARGET_T': 2.2}
Acc: 0.7185185185185186, Time: 172.7547845840454, DB size 201.37890625
{'FFT_WINDOW_SIZE': 0.1,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 4000,
 'TARGET_T': 2.2}
Acc: 0.7407407407407407, Time: 210.21229100227356, DB size 366.5859375
{'FFT_WINDOW_SIZE': 0.2,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 4000,
 'TARGET_T': 2.2}
Acc: 0.8518518518518519, Time: 203.22838521003723, DB size 347.49609375
{'FFT_WINDOW_SIZE': 0.4,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 11025,
 'TARGET_F': 4000,
 'TARGET_T': 2.2}
Acc: 0.8148148148148148, Time: 205.0415563583374, DB size 371.6328125
{'FFT_WINDOW_SIZE':

Acc: 0.8148148148148148, Time: 428.4971299171448, DB size 399.64453125
{'FFT_WINDOW_SIZE': 0.2,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 22050,
 'TARGET_F': 2000,
 'TARGET_T': 2.2}
Acc: 0.8962962962962963, Time: 420.0734758377075, DB size 343.88671875
{'FFT_WINDOW_SIZE': 0.4,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 22050,
 'TARGET_F': 2000,
 'TARGET_T': 2.2}
Acc: 0.8592592592592593, Time: 419.7284965515137, DB size 391.046875
{'FFT_WINDOW_SIZE': 0.1,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 22050,
 'TARGET_F': 4000,
 'TARGET_T': 2.2}
Acc: 0.9185185185185185, Time: 497.44073390960693, DB size 798.0703125
{'FFT_WINDOW_SIZE': 0.2,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 22050,
 'TARGET_F': 4000,
 'TARGET_T': 2.2}
Acc: 0.9333333333333333, Time: 482.87402534484863, DB size 698.234375
{'FFT_WINDOW_SIZE': 0.4,
 'POINT_EFFICIENCY': 0.8,
 'SAMPLE_RATE': 22050,
 'TARGET_F': 4000,
 'TARGET_T': 2.2}
Acc: 0.8888888888888888, Time: 486.77300810813904, DB size 774.7421875
{'FFT_WINDOW_SIZE': 0.

In [6]:
results

[({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.1},
  (508.92338037490845,
   12.44532561302185,
   0.9185185185185185,
   ['test/aha/scoundreldays1.wav',
    'test/aha/huntinghighandlow3.wav',
    'test/aha/softrainsofapril1.wav',
    'test/aha/andyoutellme1.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/takeonme6.wav',
    'test/aha/huntinghighandlow2.wav',
    'test/aha/huntinghighandlow1.wav',
    'test/ab/stillremains2.wav',
    'test/ab/wonderfullife1.wav',
    'test/ab/bleeditdry1.wav'],
   646.4375)),
 ({'SAMPLE_RATE': 22050,
   'POINT_EFFICIENCY': 0.8,
   'TARGET_T': 1.8,
   'TARGET_F': 4000,
   'FFT_WINDOW_SIZE': 0.2},
  (522.9520421028137,
   6.280886888504028,
   0.9185185185185185,
   ['test/aha/huntinghighandlow3.wav',
    'test/aha/thebluesky1.wav',
    'test/aha/theswingofthings1.wav',
    'test/aha/theswingofthings2.wav',
    'test/aha/manhattanskyline2.wav',
    'test/aha/huntinghighandlow2.wav

In [7]:
by_sample_rate = defaultdict(list)
by_point_efficiency = defaultdict(list)
by_target_t = defaultdict(list)
by_target_f = defaultdict(list)
by_fft_window_size = defaultdict(list)
by_nperseg = defaultdict(list)

for result in results:
    by_sample_rate[result[0]['SAMPLE_RATE']].append(result[1])
    by_point_efficiency[result[0]['POINT_EFFICIENCY']].append(result[1])
    by_target_t[result[0]['TARGET_T']].append(result[1])
    by_target_f[result[0]['TARGET_F']].append(result[1])
    by_fft_window_size[result[0]['FFT_WINDOW_SIZE']].append(result[1])
    by_nperseg[result[0]['FFT_WINDOW_SIZE'] * result[0]['SAMPLE_RATE']].append(result[1])

def show_stats(d):
    for value, results in d.items():
        time = np.mean([r[0] + r[1] for r in results])
        acc  = np.mean([r[2] for r in results])
        size = np.mean([r[-1] for r in results])
        print(f"{value}: T {time} A {acc} S {size} N {len(results)}")

print("Sample rate:")
show_stats(by_sample_rate)
print("Efficiency:")
show_stats(by_point_efficiency)
print("Target T:")
show_stats(by_target_t)
print("Target F:")
show_stats(by_target_f)
print("FFT Window:")
show_stats(by_fft_window_size)
print("NPERSEG:")
show_stats(by_nperseg)

Sample rate:
22050: T 465.3062574171251 A 0.801911589008363 S 660.5575856854839 N 62
44100: T 1880.893725067377 A 0.7296296296296296 S 1654.8203125 N 8
11025: T 182.12063121354137 A 0.733058984910837 S 262.22309027777777 N 54
Efficiency:
0.8: T 558.4471000625241 A 0.8229390681003582 S 648.6294732862904 N 62
1: T 754.6775241494179 A 0.30092592592592593 S 1240.16943359375 N 8
0.6: T 242.02798979370684 A 0.7724279835390947 S 337.34809027777777 N 54
Target T:
1.8: T 488.5869929898869 A 0.7496632996632997 S 566.5537997159091 N 44
2: T 492.28342108834875 A 0.7498316498316498 S 611.3244850852273 N 44
2.2: T 293.6764896247122 A 0.8100823045267491 S 459.07042100694446 N 36
Target F:
4000: T 476.25101857835597 A 0.7686868686868685 S 566.2109375 N 44
6000: T 536.2040799422698 A 0.7882154882154881 S 807.2028586647727 N 44
2000: T 255.07298641734653 A 0.739917695473251 S 220.08257378472223 N 36
FFT Window:
0.1: T 499.5755120082335 A 0.677104377104377 S 652.880859375 N 44
0.2: T 483.1778463341973 A 

In [8]:
from csv import writer

table_results = []
for r in results:
    table_results.append([r[0]['SAMPLE_RATE'], r[0]['POINT_EFFICIENCY'],
                          r[0]['TARGET_T'], r[0]['TARGET_F'], r[0]['FFT_WINDOW_SIZE'],
                          r[0]['FFT_WINDOW_SIZE']*r[0]['SAMPLE_RATE'], *r[1]])
    
with open('results.csv', 'a') as csv_file:
    w = writer(csv_file)
    w.writerows(table_results)

In [10]:
interesting_exp_parameters = [
    {
        'SAMPLE_RATE': 22050,
        'PEAK_BOX_SIZE': 25,
        'TARGET_T': 1.8,
        'TARGET_F': 5000,
        'FFT_WINDOW_SIZE': 0.1
    },
    {
        'SAMPLE_RATE': 22050,
        'PEAK_BOX_SIZE': 20,
        'POINT_EFFICIENCY': 0.8,
        'TARGET_T': 1.8,
        'TARGET_F': 5000,
        'FFT_WINDOW_SIZE': 0.1
    },
    {
        'SAMPLE_RATE': 22050,
        'PEAK_BOX_SIZE': 25,
        'POINT_EFFICIENCY': 0.6,
        'TARGET_T': 1.8,
        'TARGET_F': 5000,
        'FFT_WINDOW_SIZE': 0.1
    },
    {
        'SAMPLE_RATE': 22050,
        'PEAK_BOX_SIZE': 20,
        'POINT_EFFICIENCY': 0.6,
        'TARGET_T': 1.8,
        'TARGET_F': 5000,
        'FFT_WINDOW_SIZE': 0.1
    }
]

interesting_results = []

for params in interesting_exp_parameters:
    print(params)
    clean_for_experiment()
    result = exp_parameters, run_experiment(exp_parameters)
    print(f"Acc: {result[1][2]}, Time: {result[1][0] + result[1][1]}, DB size {result[1][-1]}")
    interesting_results.append(result)

{'SAMPLE_RATE': 22050, 'PEAK_BOX_SIZE': 25, 'TARGET_T': 1.8, 'TARGET_F': 5000, 'FFT_WINDOW_SIZE': 0.1}
Acc: 0.9259259259259259, Time: 561.7235832214355, DB size 1131.98828125
{'SAMPLE_RATE': 22050, 'PEAK_BOX_SIZE': 20, 'POINT_EFFICIENCY': 0.8, 'TARGET_T': 1.8, 'TARGET_F': 5000, 'FFT_WINDOW_SIZE': 0.1}
Acc: 0.9259259259259259, Time: 573.3089210987091, DB size 1132.11328125
{'SAMPLE_RATE': 22050, 'PEAK_BOX_SIZE': 25, 'POINT_EFFICIENCY': 0.6, 'TARGET_T': 1.8, 'TARGET_F': 5000, 'FFT_WINDOW_SIZE': 0.1}
Acc: 0.9259259259259259, Time: 571.6769626140594, DB size 1132.25390625
{'SAMPLE_RATE': 22050, 'PEAK_BOX_SIZE': 20, 'POINT_EFFICIENCY': 0.6, 'TARGET_T': 1.8, 'TARGET_F': 5000, 'FFT_WINDOW_SIZE': 0.1}
Acc: 0.9259259259259259, Time: 573.9296202659607, DB size 1132.109375
